### 두 개의 투자 전략을 하나의 class 선언

1. 클래스의 이름은 Invest 상속은 X
2. 생성자 함수
    - 매개변수
        - 주식 데이터 (_df)
        - 기준이 되는 컬럼의 이름 (_col = 'Adj Close')
        - 투자의 시작 시간 (_start = '2010-01-01')
        - 투자의 종료 시간 (_end = 'datetime.now())
    - 결측치, 양의 무한대, 음의 무한대를 제외
    - 컬럼 중 date 컬럼이 존재한다면 date 컬럼을 ㄷ인덱스로 변환
    - index를 시계열 데이터로 변환
        - 종료시간은 문자열인 경우에만 변환
    - index에 tz에 존재한다면 tz를 None으로 변환
    - 기준이 컬럼self.col, 시작시간self.start, 종료시간self.end은 self변수로 생성
3. buyandhold 함수 생성
    - 매개변수 x (self) 생성
    - 데이터프레임을 복사본 생성
    - 시작시간과 종료 시간으로 인덱스 필터, 기준이 되는 컬럼 제외하고 모두 제거
    - 일별 수익율 컬럼을 생성하여 pct_change() + 1 함수를 이용하여 값을 대입
    - 누적 수익율 컬럼을 생성하여 누적 수익율 데이터 대입
    - 만들어진 데이터프레임과 총 누적 수익율을 되돌려 준다.
4. bollinger 함수 생성
    - 매개변수 2개 
        - 신뢰구간(_num = 2)
        - 데이터 개수 (_cnt = 20)
    - 특정컬럼(self.col) 컬럼을 제외한 나머지 컬럼을 제거하고 변수에 저장
    - 이동평균선, 상단밴드, 하단밴드 생성
    - 시작시간, 종료 시간을 기준으로 데이터 필터링
    - 보유 내역 추가
5. 수익율을 계산하는 함수 생성
    - 복사본 생성
    - rtn 컬럼을 생성해서 1을 대입
    - 매수, 매도 시기를 확인하여 수익율 대입
    - rtn을 기준으로 누적 수익율을 acc_rtn 컬럼에 대입
    - 만들어진 데이터 프레임과 최종수익율을 되돌려준다.
    

In [1]:
from datetime import datetime
import pandas as pd
import numpy as np
import boll

In [2]:
import importlib
importlib.reload(boll)

<module 'boll' from 'c:\\practice_python\\6.투자전략\\boll.py'>

In [3]:
# momentum 모듈 로드
import momentum as mmt

ModuleNotFoundError: No module named 'momentum'

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
class Invest:
    def __init__(self,
                 _df,
                 _col = 'Adj Close',
                 _start = '2010-01-01',
                 _end = datetime.now()
                 ):
        self.df = _df.copy()
        self.col = _col
        self.start = _start
        self.end = _end
        
        #결측치, 양의 무한, 음의 무한 제외
        flag = self.df.isin([np.nan,np.inf,-np.inf]).any(axis=1)
        self.df = self.df.loc[~flag]
        
        # 컬럼 중 'Date'가 있다면 인덱스 재설정
        if 'Date' in self.df.columns:
            self.df.set_index('Date',inplace=True)
            
        # 인덱스 date 타임으로 변환
        self.df.index = pd.to_datetime(self.df.index, utc=True)
        
        #index에 tz 속성에 값이 존재하면 None으로 변경
        if self.df.index.tz:
            self.df.index = self.df.index.tz_localize(None)
            
        try:
            # 시작 시간은 시계열로 변경
            self.start = datetime.strptime(_start, '%Y-%m-%d')
            
            # 종료 시간이 str인 경우 datetime으로 변환
            if type(self.end) == 'str':
                self.end = datetime.strptime(self.end,'%Y,%m-%d')
        except:
            print('시간의 타입이 맞지 않습니다. 포멧은 YYYY-mm-dd 형식입니다.')
            print('Class를 다시 생성하시기 바랍니다.')

    def buyandhold(self):
        # 시작 시간, 종료 시간, 특정 컬럼을 기준으로 데이터 필터
        result = self.df.loc[self.start : self.end ,[self.col]]
        
        # 일별 수익율 
        result['rtn'] = ((result[self.col]).pct_change() +1).fillna(1)
        
        # 누적 수익율 
        result['acc_rtn'] = result['rtn'].cumprod()
        
        # 총 누적 수익율을 변수에 저장
        acc_rtn = result.iloc[-1,-1]
        return result,acc_rtn
    
    def bollinger(self,
                  _num=2,
                  _cnt = 20):
        # 특정 컬럼 제외한 나머지 컬럼 제거.
        bol_df = self.df[[self.col]].copy()
        
        # 이평선, 상단밴드, 하단밴드 생성
        bol_df['center'] = bol_df[self.col].rolling(_cnt).mean()
        bol_df['up'] = \
            bol_df['center'] + (_num * bol_df[self.col].rolling(_cnt).std())
        bol_df['down'] = \
            bol_df['center'] - (_num * bol_df[self.col].rolling(_cnt).std())
            
        # 시작시간, 종료 시간을 기준으로 데이터 필터링
        bol_df = bol_df.loc[self.start : self.end]
        
        # 보유 내역 추가
        bol_df['trade'] = ''
        
        for i in bol_df.index:
            if bol_df.loc[i,self.col] >= bol_df.loc[i,'up']:
                bol_df.loc[i,'trade'] = ''
            elif bol_df.loc[i,self.col] <= bol_df.loc[i,'down']:
                bol_df.loc[i,'trade'] = 'buy'
            else:
                if bol_df.shift().loc[i,'trade'] == 'buy':
                    bol_df.loc[i,'trade'] = 'buy'
                else:
                    bol_df.loc[i, 'trade'] = ''
        
        # class 내부의 create_rtn 함수 호출
        rtn_result, acc_rtn = self.create_rtn(bol_df)
        
        # 데이터 프레임 되돌려주기
        return rtn_result, acc_rtn
            
    def create_rtn(self,_df):
        df = _df.copy()
        
        df['rtn'] = 1
        
        for i in df.index:
            if df.shift().loc[i,'trade'] =='' and df.loc[i,'trade']=='buy':
                buy = df.loc[i,self.col]
                print(f'매수일 : {i}, 매수가 : {buy}')
            elif df.shift().loc[i,'trade'] == 'buy' and df.loc[i,'trade'] =='':
                sell = df.loc[i,self.col]
                print(f'매도일 : {i}, 매도가 : {sell}')
                # 수익률 계산
                rtn = sell/buy
                df.loc[i,'rtn'] = rtn
                print(f'수익율 : {rtn}')
        df['acc_rtn'] = df['rtn'].cumprod()
        # 총 누적 수익율을 변수에 저장
        acc_rtn = df.iloc[-1,-1]
        return df, acc_rtn
    
    # momentum 함수 생성
    # 인자 값 : 데이터 self.df, 기준이 되는 컬럼 명 self.col, 시작 시간 self.start,종료 시간 self.end, 
    #          _score=1, _momentum=12, _select=1 (self에 존재하지 않음)
    def momentum(self,
                _score = 1,
                _momentum=12,
                _select = 1
                ):
        # mmt 안에 create_ym(데이터프레임, 기준 컬럼 명) 함수 호출 -> 변수 저장    
        ym_df = mmt.create_ym(self.df,_col = self.col)
        # mmt 안에 create_month(데이터프레임, _select, _momentum,_start,_end)
        month_df = mmt.create_month(ym_df,
                                    _start = self.start,
                                    _end=self.end,
                                    _momentum = _momentum,
                                    _select =_select)
        # 거래 내역 ㅎ마수를 호출하여 변수에 저장
        # create_trade(ym_df, month_df, _score)
        trade_df = mmt.create_trade(ym_df,month_df,_score=_score)
        # 수익율 계산
        # invest class 안에 있는 create_rtn() 함수
        result, acc_rtn = self.create_rtn(trade_df)
        return result, acc_rtn
        
        
        
        
        
        

In [ ]:
df = pd.read_csv('../csv/AAPL.csv')

In [ ]:
test_invest = Invest(df)

In [ ]:
test_invest.df

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1980-12-12,0.513393,0.515625,0.513393,0.513393,0.410525,117258400.0
1980-12-15,0.488839,0.488839,0.486607,0.486607,0.389106,43971200.0
1980-12-16,0.453125,0.453125,0.450893,0.450893,0.360548,26432000.0
1980-12-17,0.462054,0.464286,0.462054,0.462054,0.369472,21610400.0
1980-12-18,0.475446,0.477679,0.475446,0.475446,0.380182,18362400.0
...,...,...,...,...,...,...
2019-06-18,196.050003,200.289993,195.210007,198.449997,198.449997,26551000.0
2019-06-19,199.679993,199.880005,197.309998,197.869995,197.869995,21124200.0
2019-06-20,200.369995,200.610001,198.029999,199.460007,199.460007,21514000.0


In [ ]:
test_invest.buyandhold()

(             Adj Close       rtn   acc_rtn
 Date                                      
 2010-01-04   26.782711  1.000000  1.000000
 2010-01-05   26.829010  1.001729  1.001729
 2010-01-06   26.402260  0.984094  0.985795
 2010-01-07   26.353460  0.998152  0.983973
 2010-01-08   26.528664  1.006648  0.990515
 ...                ...       ...       ...
 2019-06-18  198.449997  1.023518  7.409631
 2019-06-19  197.869995  0.997077  7.387975
 2019-06-20  199.460007  1.008036  7.447342
 2019-06-21  198.779999  0.996591  7.421952
 2019-06-24  199.169998  1.001962  7.436514
 
 [2384 rows x 3 columns],
 np.float64(7.436513727083075))

In [ ]:
test_invest.bollinger()

매수일 : 2010-01-22 00:00:00, 매수가 : 24.747818
매도일 : 2010-03-01 00:00:00, 매도가 : 26.154476
수익율 : 1.0568396777445188
매수일 : 2010-08-24 00:00:00, 매수가 : 30.026524
매도일 : 2010-09-08 00:00:00, 매도가 : 32.90366
수익율 : 1.0958198158401553
매수일 : 2011-03-16 00:00:00, 매수가 : 41.299767
매도일 : 2011-07-01 00:00:00, 매도가 : 42.957966
수익율 : 1.040150323366231
매수일 : 2011-11-14 00:00:00, 매수가 : 47.463268
매도일 : 2011-12-27 00:00:00, 매도가 : 50.876015
수익율 : 1.0719029081604747
매수일 : 2012-04-16 00:00:00, 매수가 : 72.601524
매도일 : 2012-06-18 00:00:00, 매도가 : 73.308609
수익율 : 1.0097392583659814
매수일 : 2012-10-08 00:00:00, 매수가 : 80.207954
매도일 : 2013-07-29 00:00:00, 매도가 : 57.243137
수익율 : 0.7136840443530077
매수일 : 2013-09-11 00:00:00, 매수가 : 60.184383
매도일 : 2013-10-18 00:00:00, 매도가 : 65.48336
수익율 : 1.0880457144505413
매수일 : 2014-01-03 00:00:00, 매수가 : 70.019096
매도일 : 2014-03-25 00:00:00, 매도가 : 70.960335
수익율 : 1.0134426042861222
매수일 : 2014-10-15 00:00:00, 매수가 : 89.842468
매도일 : 2014-10-23 00:00:00, 매도가 : 96.557182
수익율 : 1.0747387527243797
매수일 

(             Adj Close      center          up        down trade  rtn  \
 Date                                                                    
 2010-01-04   26.782711   25.037723   27.046734   23.028713        1.0   
 2010-01-05   26.829010   25.169503   27.288098   23.050908        1.0   
 2010-01-06   26.402260   25.307290   27.366449   23.248130        1.0   
 2010-01-07   26.353460   25.436879   27.410937   23.462821        1.0   
 2010-01-08   26.528664   25.525609   27.529742   23.521475        1.0   
 ...                ...         ...         ...         ...   ...  ...   
 2019-06-18  198.449997  185.432500  201.032574  169.832427   buy  1.0   
 2019-06-19  197.869995  185.996000  202.558154  169.433846   buy  1.0   
 2019-06-20  199.460007  186.830000  204.361771  169.298229   buy  1.0   
 2019-06-21  198.779999  187.786000  205.751400  169.820600   buy  1.0   
 2019-06-24  199.169998  188.796000  206.944862  170.647138   buy  1.0   
 
              acc_rtn  
 Date       

In [ ]:
test_invest.momentum()

'DatetimeIndex' object has no attribute 'dt'
매수일 : 2010-01-29 00:00:00, 매수가 : 24.035734
매도일 : 2013-02-28 00:00:00, 매도가 : 56.055256
수익율 : 2.3321632699047177
매수일 : 2013-12-31 00:00:00, 매수가 : 72.612869
매도일 : 2016-01-29 00:00:00, 매도가 : 91.581261
수익율 : 1.2612263123772178
매수일 : 2016-12-30 00:00:00, 매수가 : 111.392426
매도일 : 2019-01-31 00:00:00, 매도가 : 165.093445
수익율 : 1.4820886026847104
매수일 : 2019-03-29 00:00:00, 매수가 : 189.221313
매도일 : 2019-06-24 00:00:00, 매도가 : 199.169998
수익율 : 1.052576979000246


(             Adj Close   STD-YM trade       rtn   acc_rtn
 Date                                                     
 1980-12-12    0.410525  1980-12        1.000000  1.000000
 1980-12-15    0.389106  1980-12        1.000000  1.000000
 1980-12-16    0.360548  1980-12        1.000000  1.000000
 1980-12-17    0.369472  1980-12        1.000000  1.000000
 1980-12-18    0.380182  1980-12        1.000000  1.000000
 ...                ...      ...   ...       ...       ...
 2019-06-18  198.449997  2019-06   buy  1.000000  4.359394
 2019-06-19  197.869995  2019-06   buy  1.000000  4.359394
 2019-06-20  199.460007  2019-06   buy  1.000000  4.359394
 2019-06-21  198.779999  2019-06   buy  1.000000  4.359394
 2019-06-24  199.169998  2019-06        1.052577  4.588598
 
 [9713 rows x 5 columns],
 np.float64(4.588597970528201))

In [ ]:
import sys
list(map(
    lambda x : x.replace("\\","/"),
    sys.path
))

['c:/Users/Owner/AppData/Local/Programs/Python/Python313/python313.zip',
 'c:/Users/Owner/AppData/Local/Programs/Python/Python313/DLLs',
 'c:/Users/Owner/AppData/Local/Programs/Python/Python313/Lib',
 'c:/Users/Owner/AppData/Local/Programs/Python/Python313',
 '',
 'c:/Users/Owner/AppData/Local/Programs/Python/Python313/Lib/site-packages',
 'c:/Users/Owner/AppData/Local/Programs/Python/Python313/Lib/site-packages/win32',
 'c:/Users/Owner/AppData/Local/Programs/Python/Python313/Lib/site-packages/win32/lib',
 'c:/Users/Owner/AppData/Local/Programs/Python/Python313/Lib/site-packages/Pythonwin']